# Day 2: Black-Litterman Model with Investor Views

## Week 18 - Portfolio Optimization

---

## Learning Objectives

By the end of this notebook, you will:

1. **Understand** the Black-Litterman model and its advantages over mean-variance optimization
2. **Derive** equilibrium returns from market capitalization weights
3. **Implement** investor views (absolute and relative) in the model
4. **Calculate** posterior expected returns using Bayesian updating
5. **Build** optimal portfolios incorporating your market views

---

## Why Black-Litterman?

**Problems with Traditional Mean-Variance Optimization:**
- Highly sensitive to expected return inputs
- Often produces extreme, concentrated portfolios
- Small changes in inputs → large changes in weights
- Historical returns are poor predictors of future returns

**Black-Litterman Solution:**
- Start with equilibrium (market-implied) returns as a neutral baseline
- Blend investor views with equilibrium using Bayesian inference
- Produces more stable, intuitive portfolio weights
- Allows explicit incorporation of confidence in views

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.float_format', '{:.4f}'.format)
np.set_printoptions(precision=4, suppress=True)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries imported successfully!")

---

## Part 1: Theoretical Foundation

### 1.1 The Black-Litterman Model Framework

The Black-Litterman model combines two sources of information:

1. **Prior (Equilibrium Returns):** Market-implied expected returns derived from CAPM
2. **Views:** Investor's subjective expectations about asset returns

### Key Equations

**Equilibrium Returns (Prior):**
$$\Pi = \delta \Sigma w_{mkt}$$

Where:
- $\Pi$ = Vector of equilibrium expected excess returns
- $\delta$ = Risk aversion coefficient
- $\Sigma$ = Covariance matrix of returns
- $w_{mkt}$ = Market capitalization weights

**Investor Views:**
$$P \cdot \mu = Q + \epsilon, \quad \epsilon \sim N(0, \Omega)$$

Where:
- $P$ = Pick matrix (which assets are involved in each view)
- $Q$ = Vector of view returns
- $\Omega$ = Uncertainty matrix for views

**Posterior Expected Returns:**
$$E[R] = [(\tau\Sigma)^{-1} + P'\Omega^{-1}P]^{-1}[(\tau\Sigma)^{-1}\Pi + P'\Omega^{-1}Q]$$

**Posterior Covariance:**
$$\bar{\Sigma} = \Sigma + [(\tau\Sigma)^{-1} + P'\Omega^{-1}P]^{-1}$$

### 1.2 Parameter Definitions

| Parameter | Description | Typical Values |
|-----------|-------------|----------------|
| $\tau$ | Scaling factor for prior uncertainty | 0.01 - 0.05 |
| $\delta$ | Risk aversion coefficient | 2.5 - 3.5 |
| $\Omega$ | View uncertainty (diagonal matrix) | Proportional to $\tau \cdot P\Sigma P'$ |

**Interpretation of $\tau$:**
- Small $\tau$ → More confidence in equilibrium, views have less impact
- Large $\tau$ → Less confidence in equilibrium, views dominate

---

## Part 2: Data Collection and Preparation

In [ ]:
# Define our investment universe
# Using major sector ETFs for diversification
tickers = {
    'SPY': 'S&P 500',
    'QQQ': 'Nasdaq 100',
    'IWM': 'Russell 2000',
    'EFA': 'Intl Developed',
    'EEM': 'Emerging Markets',
    'TLT': 'Long-Term Treasury',
    'GLD': 'Gold'
}

# Download historical data
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)  # 5 years of data

print("Downloading price data...")
prices = yf.download(list(tickers.keys()), start=start_date, end=end_date)['Adj Close']
prices = prices[list(tickers.keys())]  # Ensure consistent order

print(f"\nData shape: {prices.shape}")
print(f"Date range: {prices.index[0].date()} to {prices.index[-1].date()}")
prices.tail()

In [ ]:
# Calculate returns
returns = prices.pct_change().dropna()

# Annualized statistics
annual_returns = returns.mean() * 252
annual_vol = returns.std() * np.sqrt(252)
cov_matrix = returns.cov() * 252  # Annualized covariance

# Summary statistics
stats_df = pd.DataFrame({
    'Asset': [tickers[t] for t in returns.columns],
    'Ann. Return': annual_returns.values,
    'Ann. Volatility': annual_vol.values,
    'Sharpe Ratio': (annual_returns.values / annual_vol.values)
}, index=returns.columns)

print("=" * 60)
print("ASSET STATISTICS (Annualized)")
print("=" * 60)
print(stats_df.to_string())

In [ ]:
# Visualize correlation matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Correlation heatmap
corr_matrix = returns.corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
            ax=axes[0], square=True, linewidths=0.5)
axes[0].set_title('Correlation Matrix', fontsize=12, fontweight='bold')

# Risk-Return scatter
colors = plt.cm.viridis(np.linspace(0, 1, len(tickers)))
for i, (ticker, name) in enumerate(tickers.items()):
    axes[1].scatter(annual_vol[ticker], annual_returns[ticker], 
                   s=100, c=[colors[i]], label=name, edgecolors='black')
    axes[1].annotate(ticker, (annual_vol[ticker], annual_returns[ticker]),
                    xytext=(5, 5), textcoords='offset points', fontsize=9)

axes[1].set_xlabel('Annualized Volatility', fontsize=11)
axes[1].set_ylabel('Annualized Return', fontsize=11)
axes[1].set_title('Risk-Return Profile', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper left', fontsize=8)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

---

## Part 3: Black-Litterman Implementation

### 3.1 Calculate Equilibrium Returns

In [ ]:
class BlackLitterman:
    """
    Black-Litterman Model Implementation
    
    Combines equilibrium market returns with investor views
    to generate optimal portfolio weights.
    """
    
    def __init__(self, cov_matrix, market_caps=None, risk_free_rate=0.04, 
                 delta=2.5, tau=0.05):
        """
        Initialize Black-Litterman model.
        
        Parameters:
        -----------
        cov_matrix : pd.DataFrame
            Annualized covariance matrix of asset returns
        market_caps : pd.Series, optional
            Market capitalizations for each asset
        risk_free_rate : float
            Risk-free rate (annualized)
        delta : float
            Risk aversion coefficient (typically 2.5-3.5)
        tau : float
            Scaling factor for prior uncertainty (typically 0.01-0.05)
        """
        self.cov_matrix = cov_matrix.values
        self.assets = cov_matrix.columns.tolist()
        self.n_assets = len(self.assets)
        self.risk_free_rate = risk_free_rate
        self.delta = delta
        self.tau = tau
        
        # Set market cap weights (equal weight if not provided)
        if market_caps is None:
            self.market_weights = np.ones(self.n_assets) / self.n_assets
        else:
            total_cap = market_caps.sum()
            self.market_weights = (market_caps / total_cap).values
        
        # Calculate equilibrium returns
        self.equilibrium_returns = self._calculate_equilibrium_returns()
        
        # Initialize views
        self.P = None  # Pick matrix
        self.Q = None  # View returns
        self.Omega = None  # View uncertainty
        
    def _calculate_equilibrium_returns(self):
        """
        Calculate implied equilibrium excess returns using reverse optimization.
        
        Formula: Pi = delta * Sigma * w_mkt
        """
        pi = self.delta * self.cov_matrix @ self.market_weights
        return pi
    
    def add_absolute_view(self, asset, return_view, confidence):
        """
        Add an absolute view: "Asset X will return Y%"
        
        Parameters:
        -----------
        asset : str
            Asset ticker
        return_view : float
            Expected return (annualized, decimal)
        confidence : float
            Confidence in view (0-1, where 1 is very confident)
        """
        idx = self.assets.index(asset)
        
        # Create pick row (1 for the asset, 0 elsewhere)
        p_row = np.zeros(self.n_assets)
        p_row[idx] = 1
        
        # Calculate view uncertainty
        # Lower confidence = higher uncertainty
        omega_val = (1 / confidence - 1) * self.tau * (p_row @ self.cov_matrix @ p_row)
        
        self._add_view(p_row, return_view, omega_val)
        
    def add_relative_view(self, long_asset, short_asset, return_diff, confidence):
        """
        Add a relative view: "Asset X will outperform Asset Y by Z%"
        
        Parameters:
        -----------
        long_asset : str
            Asset expected to outperform
        short_asset : str
            Asset expected to underperform
        return_diff : float
            Expected return difference (annualized, decimal)
        confidence : float
            Confidence in view (0-1)
        """
        long_idx = self.assets.index(long_asset)
        short_idx = self.assets.index(short_asset)
        
        # Create pick row (+1 for long, -1 for short)
        p_row = np.zeros(self.n_assets)
        p_row[long_idx] = 1
        p_row[short_idx] = -1
        
        # Calculate view uncertainty
        omega_val = (1 / confidence - 1) * self.tau * (p_row @ self.cov_matrix @ p_row)
        
        self._add_view(p_row, return_diff, omega_val)
        
    def _add_view(self, p_row, q_val, omega_val):
        """Internal method to add a view to the model."""
        if self.P is None:
            self.P = p_row.reshape(1, -1)
            self.Q = np.array([q_val])
            self.Omega = np.array([[omega_val]])
        else:
            self.P = np.vstack([self.P, p_row])
            self.Q = np.append(self.Q, q_val)
            # Expand Omega (diagonal matrix)
            new_omega = np.zeros((len(self.Q), len(self.Q)))
            new_omega[:-1, :-1] = self.Omega
            new_omega[-1, -1] = omega_val
            self.Omega = new_omega
            
    def clear_views(self):
        """Remove all views."""
        self.P = None
        self.Q = None
        self.Omega = None
        
    def get_posterior_returns(self):
        """
        Calculate posterior expected returns by combining
        equilibrium with views using Bayesian updating.
        
        Returns:
        --------
        np.array : Posterior expected returns
        """
        if self.P is None:
            return self.equilibrium_returns
        
        # Prior covariance scaled by tau
        tau_sigma = self.tau * self.cov_matrix
        tau_sigma_inv = np.linalg.inv(tau_sigma)
        
        # Omega inverse
        omega_inv = np.linalg.inv(self.Omega)
        
        # Posterior precision (inverse covariance)
        posterior_precision = tau_sigma_inv + self.P.T @ omega_inv @ self.P
        posterior_cov = np.linalg.inv(posterior_precision)
        
        # Posterior mean
        posterior_mean = posterior_cov @ (
            tau_sigma_inv @ self.equilibrium_returns + 
            self.P.T @ omega_inv @ self.Q
        )
        
        return posterior_mean
    
    def get_posterior_covariance(self):
        """
        Calculate posterior covariance matrix.
        
        Returns:
        --------
        np.array : Posterior covariance matrix
        """
        if self.P is None:
            return self.cov_matrix + self.tau * self.cov_matrix
        
        tau_sigma = self.tau * self.cov_matrix
        tau_sigma_inv = np.linalg.inv(tau_sigma)
        omega_inv = np.linalg.inv(self.Omega)
        
        posterior_precision = tau_sigma_inv + self.P.T @ omega_inv @ self.P
        M = np.linalg.inv(posterior_precision)
        
        return self.cov_matrix + M
    
    def optimize_portfolio(self, risk_aversion=None, constraints=None):
        """
        Calculate optimal portfolio weights using posterior estimates.
        
        Parameters:
        -----------
        risk_aversion : float, optional
            Risk aversion for optimization (uses self.delta if None)
        constraints : dict, optional
            Additional constraints for optimization
            
        Returns:
        --------
        dict : Optimal weights and portfolio statistics
        """
        if risk_aversion is None:
            risk_aversion = self.delta
            
        mu = self.get_posterior_returns()
        sigma = self.get_posterior_covariance()
        
        # Objective: maximize utility = mu'w - (delta/2) * w'Σw
        def neg_utility(w):
            return -(w @ mu - (risk_aversion / 2) * w @ sigma @ w)
        
        # Constraints
        cons = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]  # Weights sum to 1
        
        # Bounds (0-1 for long-only, or allow shorts)
        if constraints and constraints.get('long_only', True):
            bounds = [(0, 1) for _ in range(self.n_assets)]
        else:
            bounds = [(-1, 2) for _ in range(self.n_assets)]
        
        # Initial guess
        w0 = np.ones(self.n_assets) / self.n_assets
        
        # Optimize
        result = minimize(neg_utility, w0, method='SLSQP', 
                         bounds=bounds, constraints=cons)
        
        optimal_weights = result.x
        
        # Calculate portfolio statistics
        port_return = optimal_weights @ mu
        port_vol = np.sqrt(optimal_weights @ sigma @ optimal_weights)
        port_sharpe = (port_return - self.risk_free_rate) / port_vol
        
        return {
            'weights': pd.Series(optimal_weights, index=self.assets),
            'expected_return': port_return,
            'volatility': port_vol,
            'sharpe_ratio': port_sharpe
        }
    
    def get_summary(self):
        """Print model summary."""
        print("=" * 60)
        print("BLACK-LITTERMAN MODEL SUMMARY")
        print("=" * 60)
        
        print(f"\nParameters:")
        print(f"  Risk Aversion (δ): {self.delta}")
        print(f"  Tau (τ): {self.tau}")
        print(f"  Risk-free Rate: {self.risk_free_rate:.2%}")
        
        print(f"\nEquilibrium Returns:")
        for i, asset in enumerate(self.assets):
            print(f"  {asset}: {self.equilibrium_returns[i]:.2%}")
            
        if self.P is not None:
            print(f"\nNumber of Views: {len(self.Q)}")
            posterior = self.get_posterior_returns()
            print(f"\nPosterior Returns:")
            for i, asset in enumerate(self.assets):
                diff = posterior[i] - self.equilibrium_returns[i]
                print(f"  {asset}: {posterior[i]:.2%} (Δ {diff:+.2%})")
        else:
            print("\nNo views added yet.")

In [ ]:
# Create approximate market cap weights for our ETFs
# These are illustrative - in practice, use actual market caps
market_caps = pd.Series({
    'SPY': 500,   # S&P 500 - largest
    'QQQ': 200,   # Nasdaq
    'IWM': 50,    # Small caps
    'EFA': 100,   # International developed
    'EEM': 40,    # Emerging markets
    'TLT': 30,    # Treasuries
    'GLD': 80     # Gold
})  # In billions USD (illustrative)

# Initialize Black-Litterman model
bl_model = BlackLitterman(
    cov_matrix=cov_matrix,
    market_caps=market_caps,
    risk_free_rate=0.04,  # 4% risk-free rate
    delta=2.5,            # Risk aversion
    tau=0.05              # Prior uncertainty
)

# Display equilibrium returns
bl_model.get_summary()

### 3.2 Compare Equilibrium vs Historical Returns

In [ ]:
# Compare equilibrium returns with historical returns
comparison_df = pd.DataFrame({
    'Asset': list(tickers.values()),
    'Historical Return': annual_returns.values,
    'Equilibrium Return': bl_model.equilibrium_returns,
    'Difference': annual_returns.values - bl_model.equilibrium_returns
}, index=list(tickers.keys()))

print("Historical vs Equilibrium Returns")
print("=" * 60)
print(comparison_df.to_string())

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(tickers))
width = 0.35

bars1 = ax.bar(x - width/2, annual_returns.values, width, label='Historical', color='steelblue')
bars2 = ax.bar(x + width/2, bl_model.equilibrium_returns, width, label='Equilibrium (BL)', color='coral')

ax.set_xlabel('Asset')
ax.set_ylabel('Annualized Return')
ax.set_title('Historical vs Black-Litterman Equilibrium Returns', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(list(tickers.keys()))
ax.legend()
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

plt.tight_layout()
plt.show()

---

## Part 4: Incorporating Investor Views

### 4.1 Types of Views

**Absolute Views:**
- "Tech stocks (QQQ) will return 15% next year"
- "Gold (GLD) will return 8%"

**Relative Views:**
- "US stocks (SPY) will outperform Emerging Markets (EEM) by 5%"
- "Small caps (IWM) will underperform large caps (SPY) by 3%"

In [ ]:
# Create a fresh model for our views
bl_with_views = BlackLitterman(
    cov_matrix=cov_matrix,
    market_caps=market_caps,
    risk_free_rate=0.04,
    delta=2.5,
    tau=0.05
)

# Add our investment views
print("Adding Investor Views:")
print("=" * 60)

# View 1: Absolute - Bullish on Tech
# "QQQ will return 15% over the next year"
bl_with_views.add_absolute_view(
    asset='QQQ',
    return_view=0.15,   # 15% expected return
    confidence=0.7      # 70% confident
)
print("View 1: QQQ will return 15% (70% confidence)")

# View 2: Relative - US > EM
# "SPY will outperform EEM by 5%"
bl_with_views.add_relative_view(
    long_asset='SPY',
    short_asset='EEM',
    return_diff=0.05,   # 5% outperformance
    confidence=0.8      # 80% confident
)
print("View 2: SPY will outperform EEM by 5% (80% confidence)")

# View 3: Absolute - Bearish on Bonds in rising rate environment
bl_with_views.add_absolute_view(
    asset='TLT',
    return_view=0.02,   # Only 2% expected return
    confidence=0.6      # 60% confident
)
print("View 3: TLT will return only 2% (60% confidence)")

# View 4: Relative - Gold as hedge
bl_with_views.add_relative_view(
    long_asset='GLD',
    short_asset='TLT',
    return_diff=0.04,   # Gold outperforms bonds by 4%
    confidence=0.65
)
print("View 4: GLD will outperform TLT by 4% (65% confidence)")

In [ ]:
# Display the Pick Matrix (P) and Views (Q)
print("\nPick Matrix (P):")
print("=" * 60)
P_df = pd.DataFrame(bl_with_views.P, 
                    columns=bl_with_views.assets,
                    index=[f'View {i+1}' for i in range(len(bl_with_views.Q))])
print(P_df.to_string())

print("\nView Returns (Q):")
for i, q in enumerate(bl_with_views.Q):
    print(f"  View {i+1}: {q:.2%}")

print("\nView Uncertainty Matrix (Ω) - Diagonal:")
for i in range(len(bl_with_views.Q)):
    print(f"  View {i+1}: {bl_with_views.Omega[i,i]:.6f}")

In [ ]:
# Get and display posterior returns
bl_with_views.get_summary()

In [ ]:
# Visualize the impact of views
equilibrium = bl_model.equilibrium_returns
posterior = bl_with_views.get_posterior_returns()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart comparison
x = np.arange(len(tickers))
width = 0.35

axes[0].bar(x - width/2, equilibrium, width, label='Equilibrium (Prior)', color='lightblue', edgecolor='steelblue')
axes[0].bar(x + width/2, posterior, width, label='Posterior (w/ Views)', color='coral', edgecolor='darkred')
axes[0].set_xlabel('Asset')
axes[0].set_ylabel('Expected Return')
axes[0].set_title('Impact of Investor Views on Expected Returns', fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(list(tickers.keys()))
axes[0].legend()
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

# Change in returns
changes = posterior - equilibrium
colors = ['green' if c > 0 else 'red' for c in changes]
axes[1].barh(list(tickers.keys()), changes, color=colors, edgecolor='black')
axes[1].set_xlabel('Change in Expected Return')
axes[1].set_title('Change from Equilibrium Due to Views', fontweight='bold')
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[1].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.1%}'.format(x)))

plt.tight_layout()
plt.show()

---

## Part 5: Portfolio Optimization

### 5.1 Optimal Weights with and without Views

In [ ]:
# Optimize portfolios
# 1. Market Cap Weights (starting point)
market_weights = market_caps / market_caps.sum()

# 2. Equilibrium (no views)
equil_portfolio = bl_model.optimize_portfolio()

# 3. With views
views_portfolio = bl_with_views.optimize_portfolio()

# Compare weights
weights_comparison = pd.DataFrame({
    'Market Cap Weights': market_weights.values,
    'Equilibrium (No Views)': equil_portfolio['weights'].values,
    'With Investor Views': views_portfolio['weights'].values
}, index=list(tickers.keys()))

print("Portfolio Weight Comparison")
print("=" * 60)
print(weights_comparison.round(4).to_string())

print("\n" + "=" * 60)
print("Portfolio Statistics")
print("=" * 60)
stats_comparison = pd.DataFrame({
    'Equilibrium (No Views)': [
        equil_portfolio['expected_return'],
        equil_portfolio['volatility'],
        equil_portfolio['sharpe_ratio']
    ],
    'With Investor Views': [
        views_portfolio['expected_return'],
        views_portfolio['volatility'],
        views_portfolio['sharpe_ratio']
    ]
}, index=['Expected Return', 'Volatility', 'Sharpe Ratio'])
print(stats_comparison.round(4).to_string())

In [ ]:
# Visualize portfolio weights
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Color palette
colors = plt.cm.Set3(np.linspace(0, 1, len(tickers)))

# Market cap weights
axes[0].pie(market_weights, labels=list(tickers.keys()), autopct='%1.1f%%',
           colors=colors, startangle=90)
axes[0].set_title('Market Cap Weights', fontweight='bold')

# Equilibrium weights
eq_weights = equil_portfolio['weights'].values
eq_weights_clean = np.maximum(eq_weights, 0)  # Handle small negatives
axes[1].pie(eq_weights_clean, labels=list(tickers.keys()), autopct='%1.1f%%',
           colors=colors, startangle=90)
axes[1].set_title('Equilibrium (No Views)', fontweight='bold')

# With views
view_weights = views_portfolio['weights'].values
view_weights_clean = np.maximum(view_weights, 0)
axes[2].pie(view_weights_clean, labels=list(tickers.keys()), autopct='%1.1f%%',
           colors=colors, startangle=90)
axes[2].set_title('With Investor Views', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Weight change analysis
weight_changes = views_portfolio['weights'] - equil_portfolio['weights']

fig, ax = plt.subplots(figsize=(10, 6))
colors = ['green' if w > 0 else 'red' for w in weight_changes]
bars = ax.barh(list(tickers.keys()), weight_changes * 100, color=colors, edgecolor='black')

ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_xlabel('Weight Change (%)', fontsize=11)
ax.set_title('Portfolio Weight Changes Due to Investor Views', fontsize=12, fontweight='bold')

# Add value labels
for bar, val in zip(bars, weight_changes * 100):
    if val >= 0:
        ax.text(val + 0.3, bar.get_y() + bar.get_height()/2, f'+{val:.1f}%',
               va='center', fontsize=10)
    else:
        ax.text(val - 0.3, bar.get_y() + bar.get_height()/2, f'{val:.1f}%',
               va='center', ha='right', fontsize=10)

plt.tight_layout()
plt.show()

---

## Part 6: Sensitivity Analysis

### 6.1 Impact of Confidence Levels

In [ ]:
# Analyze how confidence affects posterior returns
confidence_levels = [0.3, 0.5, 0.7, 0.9, 0.99]
qqq_posteriors = []
spy_posteriors = []

for conf in confidence_levels:
    temp_model = BlackLitterman(
        cov_matrix=cov_matrix,
        market_caps=market_caps,
        risk_free_rate=0.04,
        delta=2.5,
        tau=0.05
    )
    
    # Add bullish QQQ view with varying confidence
    temp_model.add_absolute_view('QQQ', 0.15, conf)
    
    posterior = temp_model.get_posterior_returns()
    qqq_idx = temp_model.assets.index('QQQ')
    spy_idx = temp_model.assets.index('SPY')
    qqq_posteriors.append(posterior[qqq_idx])
    spy_posteriors.append(posterior[spy_idx])

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(confidence_levels, qqq_posteriors, 'o-', color='blue', linewidth=2, 
        markersize=8, label='QQQ (view asset)')
ax.plot(confidence_levels, spy_posteriors, 's--', color='gray', linewidth=2,
        markersize=8, label='SPY (no direct view)')

# Reference lines
ax.axhline(y=bl_model.equilibrium_returns[bl_model.assets.index('QQQ')], 
          color='blue', linestyle=':', alpha=0.5, label='QQQ Equilibrium')
ax.axhline(y=0.15, color='green', linestyle=':', alpha=0.5, label='View (15%)')

ax.set_xlabel('Confidence Level', fontsize=11)
ax.set_ylabel('Posterior Expected Return', fontsize=11)
ax.set_title('Impact of View Confidence on Posterior Returns\n(View: QQQ = 15%)', 
            fontsize=12, fontweight='bold')
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
ax.set_xticks(confidence_levels)

plt.tight_layout()
plt.show()

print("\nKey Insight: Higher confidence pulls posterior closer to the view.")
print("At low confidence, posterior stays near equilibrium.")

### 6.2 Impact of Tau (Prior Uncertainty)

In [ ]:
# Analyze how tau affects results
tau_values = [0.01, 0.025, 0.05, 0.1, 0.25]
qqq_posteriors_tau = []

for tau in tau_values:
    temp_model = BlackLitterman(
        cov_matrix=cov_matrix,
        market_caps=market_caps,
        risk_free_rate=0.04,
        delta=2.5,
        tau=tau  # Varying tau
    )
    
    temp_model.add_absolute_view('QQQ', 0.15, 0.7)  # Same view
    
    posterior = temp_model.get_posterior_returns()
    qqq_idx = temp_model.assets.index('QQQ')
    qqq_posteriors_tau.append(posterior[qqq_idx])

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(tau_values, qqq_posteriors_tau, 'o-', color='purple', linewidth=2, markersize=8)
ax.axhline(y=bl_model.equilibrium_returns[bl_model.assets.index('QQQ')], 
          color='blue', linestyle=':', alpha=0.7, label='Equilibrium')
ax.axhline(y=0.15, color='green', linestyle=':', alpha=0.7, label='View (15%)')

ax.set_xlabel('Tau (τ)', fontsize=11)
ax.set_ylabel('QQQ Posterior Expected Return', fontsize=11)
ax.set_title('Impact of Tau on Posterior Returns\n(Higher τ = Less confidence in equilibrium)', 
            fontsize=12, fontweight='bold')
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

plt.tight_layout()
plt.show()

print("\nKey Insight: Higher tau means less trust in equilibrium,")
print("so views have more impact on the posterior.")

---

## Part 7: Practical Considerations

### 7.1 Common Pitfalls and Best Practices

In [ ]:
# Demonstrate conflicting views
print("Example: What happens with conflicting views?")
print("=" * 60)

conflict_model = BlackLitterman(
    cov_matrix=cov_matrix,
    market_caps=market_caps,
    risk_free_rate=0.04,
    delta=2.5,
    tau=0.05
)

# Conflicting views: QQQ bullish but also expect it to underperform SPY
conflict_model.add_absolute_view('QQQ', 0.20, 0.8)  # Very bullish on QQQ
conflict_model.add_relative_view('SPY', 'QQQ', 0.05, 0.7)  # But SPY > QQQ?

posterior = conflict_model.get_posterior_returns()

print("\nViews Added:")
print("  1. QQQ will return 20% (80% confidence)")
print("  2. SPY will outperform QQQ by 5% (70% confidence)")
print("\nThese views are potentially conflicting!")
print("\nPosterior Returns:")
for i, asset in enumerate(conflict_model.assets):
    eq = conflict_model.equilibrium_returns[i]
    print(f"  {asset}: {posterior[i]:.2%} (equilibrium: {eq:.2%})")

print("\n→ The model balances conflicting views based on confidence levels.")

In [ ]:
# Best practices summary
best_practices = """
╔══════════════════════════════════════════════════════════════════╗
║              BLACK-LITTERMAN BEST PRACTICES                      ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  1. START WITH GOOD EQUILIBRIUM                                  ║
║     • Use market cap weights from reliable sources               ║
║     • Ensure covariance matrix is well-conditioned              ║
║     • Consider using shrinkage estimators for covariance        ║
║                                                                  ║
║  2. FORMULATE VIEWS CAREFULLY                                    ║
║     • Views should be forward-looking, not backward-looking     ║
║     • Express views you actually have conviction in             ║
║     • Relative views are often more reliable than absolute      ║
║                                                                  ║
║  3. CALIBRATE CONFIDENCE APPROPRIATELY                           ║
║     • High confidence (>80%) only for very strong views         ║
║     • Default to moderate confidence (50-70%)                   ║
║     • Avoid extreme confidence (>95%) - rarely justified        ║
║                                                                  ║
║  4. CHOOSE TAU WISELY                                           ║
║     • τ = 0.025 to 0.05 is typical                              ║
║     • Lower τ → more weight on equilibrium                       ║
║     • Some practitioners set τ = 1/T (T = sample size)          ║
║                                                                  ║
║  5. VALIDATE RESULTS                                            ║
║     • Check that weights are sensible                           ║
║     • Perform sensitivity analysis                              ║
║     • Compare to benchmark allocations                          ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
"""
print(best_practices)

---

## Part 8: Interview Questions & Key Takeaways

### Common Interview Questions

In [ ]:
interview_questions = {
    "Q1: What problem does Black-Litterman solve?": """
    A: Black-Litterman addresses the extreme sensitivity of mean-variance 
    optimization to expected return inputs. Traditional MVO often produces 
    concentrated, unstable portfolios because small changes in return estimates 
    lead to large weight changes. BL solves this by:
    1. Starting with a neutral equilibrium (market-implied returns)
    2. Allowing investors to express views with uncertainty
    3. Using Bayesian inference to blend views with equilibrium
    """,
    
    "Q2: How are equilibrium returns calculated?": """
    A: Equilibrium returns (Π) are derived through "reverse optimization":
       Π = δ × Σ × w_mkt
    Where:
    - δ = risk aversion coefficient (typically 2.5)
    - Σ = covariance matrix of returns
    - w_mkt = market capitalization weights
    
    This gives the implied returns that would make the market portfolio optimal.
    """,
    
    "Q3: What's the difference between absolute and relative views?": """
    A: 
    Absolute View: "Asset X will return Y%"
       - Pick matrix has 1 in asset position, 0 elsewhere
       - Example: "Tech stocks will return 12%"
    
    Relative View: "Asset X will outperform Asset Y by Z%"
       - Pick matrix has +1 for long asset, -1 for short
       - Example: "US equities will beat emerging markets by 5%"
       - Often more reliable as they cancel out common factors
    """,
    
    "Q4: What role does τ (tau) play in the model?": """
    A: Tau (τ) represents uncertainty in the equilibrium returns (prior):
    - Small τ (0.01): Strong belief in equilibrium, views have less impact
    - Large τ (0.10+): Weaker belief in equilibrium, views dominate
    - Typical range: 0.025 to 0.05
    - Some set τ = 1/T where T = number of observations
    - It scales the prior covariance matrix (τΣ)
    """,
    
    "Q5: How does confidence in views affect the results?": """
    A: View confidence is expressed through the Ω matrix:
    - High confidence → small Ω → views pull posterior strongly toward Q
    - Low confidence → large Ω → views have minimal impact
    - Common formula: ω_i = (1/conf - 1) × τ × P_i × Σ × P_i'
    
    At 100% confidence, posterior equals the view exactly.
    At 0% confidence, posterior equals equilibrium.
    """,
    
    "Q6: What are limitations of Black-Litterman?": """
    A: Key limitations include:
    1. Still requires accurate covariance estimation
    2. Assumes normal distribution of returns
    3. Single-period model (ignores dynamics)
    4. Views are subjective - garbage in, garbage out
    5. Doesn't account for transaction costs or constraints
    6. Equilibrium assumption may not hold in crisis periods
    """
}

for question, answer in interview_questions.items():
    print("\n" + "=" * 70)
    print(question)
    print("=" * 70)
    print(answer)

### Key Formulas to Remember

| Formula | Description |
|---------|-------------|
| $\Pi = \delta \Sigma w_{mkt}$ | Equilibrium returns |
| $E[R] = [(\tau\Sigma)^{-1} + P'\Omega^{-1}P]^{-1}[(\tau\Sigma)^{-1}\Pi + P'\Omega^{-1}Q]$ | Posterior returns |
| $\omega_i = \frac{1-c}{c} \cdot \tau \cdot P_i \Sigma P_i'$ | View uncertainty (c = confidence) |
| $w^* = \frac{1}{\delta}\Sigma^{-1}\mu$ | Optimal weights (unconstrained) |

---

## Summary

### What We Covered Today

1. **Black-Litterman Framework**: Understanding how it combines equilibrium with investor views

2. **Equilibrium Returns**: Deriving market-implied returns using reverse optimization

3. **Investor Views**: 
   - Absolute views ("Asset X returns Y%")
   - Relative views ("A outperforms B by Z%")
   - Expressing confidence through Ω matrix

4. **Posterior Calculation**: Bayesian updating to blend prior and views

5. **Sensitivity Analysis**: Impact of τ and confidence on results

### Key Takeaways

- Black-Litterman produces more **stable, intuitive** portfolios than traditional MVO
- The model **requires views** - with no views, you get market-cap weights
- **Confidence calibration** is crucial - be honest about uncertainty
- **Relative views** are often more reliable than absolute views
- Always perform **sensitivity analysis** before implementing

---

### Next Steps

- Day 3: Risk Parity and Equal Risk Contribution portfolios
- Day 4: Hierarchical Risk Parity (HRP)
- Day 5: Transaction costs and portfolio rebalancing

In [ ]:
# Final summary visualization
print("\n" + "="*70)
print("FINAL PORTFOLIO COMPARISON")
print("="*70)

final_comparison = pd.DataFrame({
    'Market Cap': market_weights.values,
    'Equilibrium BL': equil_portfolio['weights'].values,
    'BL + Views': views_portfolio['weights'].values
}, index=list(tickers.keys()))

print("\nWeights:")
print(final_comparison.round(3).to_string())

print("\nPortfolio Metrics (BL with Views):")
print(f"  Expected Return: {views_portfolio['expected_return']:.2%}")
print(f"  Volatility:      {views_portfolio['volatility']:.2%}")
print(f"  Sharpe Ratio:    {views_portfolio['sharpe_ratio']:.2f}")

print("\n✅ Day 2 Complete: Black-Litterman Model with Investor Views")